## **Build a Web Research Agent**

### **Introduction**

In the previous excercise, you learned the basics of the Tavily API. Now, let's take it a step further: we'll combine that knowledge with LLMs to unlock real value. In this tutorial, you'll learn how to build a web research agent that can search, extract, crawl and reason over live web data.

You already explored the various parameter configurations available for each Tavily API endpoint. With the official `Tavily-LangChain integration`, our agent can automatically set these parameters - like `time_range` for search or specific crawl `instructions` - based on the context and requirements of each task. This dynamic, LLM-powered configuration is powerful in agentic systems.

By the end of this lesson, you'll know how to:
- Seamlessly connect foundation models to the web for up-to-date research
- Build a react-style web agent
- Dynamically configure search, extract and crawl parameters the Tavily-LangChain integration

### **Getting Started** 

In [1]:
# install dependencies
!uv add tavily-python langchain-google-genai langchain langchain-tavily langgraph python-dotenv --quiet

### **Setting Up**

In [2]:
import os
from dotenv import load_dotenv
from tavily import TavilyClient

load_dotenv()

tavily_api_key = os.getenv("TAVILY_API_KEY")
gg_api_key = os.getenv("GOOGLE_API_KEY")

if tavily_api_key is None or gg_api_key is None:
    print("Please set API Key")
else:
    tavily_client = TavilyClient(api_key=tavily_api_key)
    print("Setup succesfully")

Setup succesfully


Let's define the following modular tools with the Tavily-LangChain integration:
1. **Search** the web for relevant information
2. **Extract** content from specific web pages
3. **Crawl** entire websites

In [3]:
# Define the set of web tools our agent will use to interact with the Tavily API
from langchain_tavily import TavilySearch, TavilyCrawl, TavilyExtract

search = TavilySearch(max_results=5, topic="general")
extract = TavilyExtract(extract_depth="basic")
crawl = TavilyCrawl()

Initialize the LLM Agent

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    api_key = gg_api_key
)

# test setup
response = llm.invoke("Hi, how are you?")
print(response.content)

Hi there! I'm doing well, thank you for asking.

How about you? How are things going on your end?


### **Web Agent**

Next, we'll build a Web Agent powered by Tavily, which consists of three main components: the language model, a set of web tools, and a system prompt. The language model serves as the agent's "brain", while the web tools (Search, Extract and Crawl) allow the agent to interact with and gather information from the internet. The system prompt guides the agent's behavior, explaining how and when to use each tool to accomplish its research goals.

This agent leverages a pre-built LangGraph reAct implementation, as illusetrated in the diagram below. The reAct framework enables the agent to reason about which actions to take, use the web tools in sequence, and iterate as needed until it completes its research task. The system prompt is especially important - it instructs the agent on best pratices for using the tools together, ensuring that the agent's response are thorough, accurate and well-sourced.

You are encouraged to experiment with the system prompt or try different language models to change the agent's style, personality, or optimize its performance for specific use cases.

In [5]:
import datetime
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

today = datetime.datetime.today().strftime("%A, %B %d, %Y")

system_prompt = f"""    
        You are a research agent equipped with advanced web tools: Tavily Web Search, Web Crawl, and Web Extract. Your mission is to conduct comprehensive, accurate, and up-to-date research, grounding your findings in credible web sources.

        **Today's Date:** {today}

        **Available Tools:**

        1. **Tavily Web Search**

        * **Purpose:** Retrieve relevant web pages based on a query.
        * **Usage:** Provide a search query to receive semantically ranked results, each containing the title, URL, and a content snippet.
        * **Best Practices:**

            * Use specific queries to narrow down results.
            * Optimize searches using parameters such as `search_depth`, `time_range`, `include_domains`, and `include_raw_content`.
            * Break down complex queries into specific, focused sub-queries.

        2. **Tavily Web Crawl**

        * **Purpose:** Explore a website's structure and gather content from linked pages for deep research and information discovery from a single source.
        * **Usage:** Input a base URL to crawl, specifying parameters such as `max_depth`, `max_breadth`, and `extract_depth`.
        * **Best Practices:**

            * Begin with shallow crawls and progressively increase depth.
            * Utilize `select_paths` or `exclude_paths` to focus the crawl.
            * Set `extract_depth` to "advanced" for comprehensive extraction.

        3. **Tavily Web Extract**

        * **Purpose:** Extract the full content from specific web pages.
        * **Usage:** Provide URLs to retrieve detailed content.
        * **Best Practices:**

            * Set `extract_depth` to "advanced" for detailed content, including tables and embedded media.
            * Enable `include_images` if image data is necessary.

        **Guidelines for Conducting Research:**

        * **Citations:** Always support findings with source URLs, clearly provided as in-text citations.
        * **Accuracy:** Rely solely on data obtained via provided tools—never fabricate information.
        * **Methodology:** Follow a structured approach:

        * **Thought:** Consider necessary information and next steps.
        * **Action:** Select and execute appropriate tools.
        * **Observation:** Analyze obtained results.
        * Repeat Thought/Action/Observation cycles as needed.
        * **Final Answer:** Synthesize and present findings with citations in markdown format.

        **Example Workflows:**

        **Workflow 1: Search Only**

        **Question:** What are recent news headlines about artificial intelligence?

        * **Thought:** I need quick, recent articles about AI.
        * **Action:** Use Tavily Web Search with the query "recent artificial intelligence news" and set `time_range` to "week".
        * **Observation:** Retrieved 10 relevant articles from reputable news sources.
        * **Final Answer:** Summarize key headlines with citations.

        **Workflow 2: Search and Extract**

        **Question:** Provide detailed insights into recent advancements in quantum computing.

        * **Thought:** I should find recent detailed articles first.
        * **Action:** Use Tavily Web Search with the query "recent advancements in quantum computing" and set `time_range` to "month".
        * **Observation:** Retrieved 10 relevant results.
        * **Thought:** I should extract content from the most comprehensive article.
        * **Action:** Use Tavily Web Extract on the most relevant URL from search results.
        * **Observation:** Extracted detailed information about quantum computing advancements.
        * **Final Answer:** Provide detailed insights summarized from extracted content with citations.

        **Workflow 3: Search and Crawl**

        **Question:** What are the latest advancements in renewable energy technologies?

        * **Thought:** I need recent articles about advancements in renewable energy.
        * **Action:** Use Tavily Web Search with the query "latest advancements in renewable energy technologies" and set `time_range` to "month".
        * **Observation:** Retrieved 10 articles discussing recent developments in solar panels, wind turbines, and energy storage.
        * **Thought:** To gain deeper insights, I'll crawl a relevant industry-leading renewable energy site.
        * **Action:** Use Tavily Web Crawl on the URL of a leading renewable energy industry website, setting `max_depth` to 2.
        * **Observation:** Gathered extensive content from multiple articles linked on the site, highlighting new technologies and innovations.
        * **Final Answer:** Provide a synthesized summary of findings with citations.

        ---

        You will now receive a research question from the user:

        """

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        MessagesPlaceholder(variable_name="messages")
    ],
    
)

# Initialize the web agent
web_agent = create_react_agent(
    model = llm,
    tools = [search, extract, crawl],
    prompt = prompt,
    name = "web_agent"
)

### **Test web agent**

In [6]:
from langchain.schema import HumanMessage

inputs = {
    "messages": [
        HumanMessage(
            content = "find all the iphone models currently available on apple.com and their prices"
        )
    ]
}

for s in web_agent.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

find all the iphone models currently available on apple.com and their prices
================================== Ai Message ==================================
Name: web_agent
Tool Calls:
  tavily_crawl (f3b62d1c-ca10-4114-8bca-9df75d59200d)
 Call ID: f3b62d1c-ca10-4114-8bca-9df75d59200d
  Args:
    instructions: Find all iPhone models and their prices.
    max_depth: 2
    select_paths: ['/iphone.*']
    extract_depth: advanced
    url: https://www.apple.com
================================= Tool Message =================================
Name: tavily_crawl

{"base_url": "https://www.apple.com", "results": [{"url": "https://www.apple.com/iphone", "raw_content": "Get credit toward iPhone 17, iPhone Air, or iPhone 17 Pro when you trade in an eligible smartphone.Get credit toward iPhone 17, iPhone Air, or iPhone 17 Pro when you trade in an eligible smartphone.[\\*](#footnote-1) [Shop iPhone](/us/shop/goto/buy_

In [7]:
from IPython.display import Markdown

Markdown(message.content)

The following iPhone models are currently available on apple.com:

*   **iPhone 17 Pro**
*   **iPhone 17 Pro Max**
*   **iPhone Air**
*   **iPhone 17**
*   **iPhone 16e**
*   **iPhone 16 Pro**
*   **iPhone 16 Pro Max**
*   **iPhone 16**
*   **iPhone 16 Plus**
*   **iPhone 15 Pro**
*   **iPhone 15 Pro Max**
*   **iPhone 15**
*   **iPhone 15 Plus**
*   **iPhone 14 Pro**
*   **iPhone 14 Pro Max**
*   **iPhone 14**
*   **iPhone 14 Plus**
*   **iPhone 13 Pro**
*   **iPhone 13 Pro Max**
*   **iPhone 13 mini**
*   **iPhone 13**
*   **iPhone SE (3rd generation)**
*   **iPhone 12 Pro**
*   **iPhone 12 Pro Max**
*   **iPhone 12 mini**
*   **iPhone 12**
*   **iPhone 11 Pro**
*   **iPhone 11 Pro Max**
*   **iPhone 11**
*   **iPhone SE (2nd generation)**
*   **iPhone XS**
*   **iPhone XS Max**
*   **iPhone XR**
*   **iPhone X**
*   **iPhone 8 Plus**
*   **iPhone 8**
*   **iPhone 7 Plus**
*   **iPhone 7**

The prices are listed as "From $price.display.smart or $price.display.perMonth for $price.display.months mo." on the comparison page, indicating that specific numerical prices are dynamically loaded or depend on configuration and carrier options [https://www.apple.com/iphone/compare]. However, the main iPhone page mentions that the iPhone 17 includes a $30 connectivity discount [https://www.apple.com/iphone]. The iPhone 16 and iPhone 16 Plus also include a $30 connectivity discount for Boost Mobile, T-Mobile, and Verizon customers [https://www.apple.com/iphone].

For the iPhone 16e, the page highlights "Everything you love. Including the price." but doesn't provide a specific numerical price [https://www.apple.com/iphone-16e].

In [8]:
inputs = {
    "messages": [
        HumanMessage(
            content="return 5 job postings for a software engineer in the bay area on linkedin"
        )
    ]
}

# Stream the web agent's response
for s in web_agent.stream(inputs, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

return 5 job postings for a software engineer in the bay area on linkedin
================================== Ai Message ==================================
Name: web_agent
Tool Calls:
  tavily_search (b48b50fb-384a-404b-862d-eecd90829e3c)
 Call ID: b48b50fb-384a-404b-862d-eecd90829e3c
  Args:
    include_domains: ['linkedin.com']
    query: 5 software engineer job postings in Bay Area on LinkedIn
    topic: careers
================================= Tool Message =================================
Name: tavily_search

Error: 1 validation error for TavilySearchInput
topic
  Input should be 'general', 'news' or 'finance' [type=literal_error, input_value='careers', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
 Please fix your mistakes.
================================== Ai Message ==================================
Name: web_agent
Tool Calls:
  tavily_search 

In [9]:
Markdown(message.content)

Here are 5 software engineer job postings in the San Francisco Bay Area from LinkedIn:

1.  **Software Engineer, AI Platform** at LinkedIn in Mountain View, CA. The salary range is $160,000 - $180,000. [https://www.linkedin.com/jobs/view/software-engineer-ai-platform-at-linkedin-4315367413](https://www.linkedin.com/jobs/view/software-engineer-ai-platform-at-linkedin-4315367413)
2.  **Software Engineer, Backend** (multiple postings) [https://www.linkedin.com/jobs/software-engineer-jobs-san-francisco-bay-area](https://www.linkedin.com/jobs/software-engineer-jobs-san-francisco-bay-area)
3.  **Software Engineer (New Grad)** [https://www.linkedin.com/jobs/software-engineer-jobs-san-francisco-bay-area](https://www.linkedin.com/jobs/software-engineer-jobs-san-francisco-bay-area)
4.  **Software Development Engineer I - Frontend & Mobile** [https://www.linkedin.com/jobs/software-engineer-jobs-san-francisco-bay-area](https://www.linkedin.com/jobs/software-engineer-jobs-san-francisco-bay-area)
5.  **Software Engineer, Developer Experience** [https://www.linkedin.com/jobs/software-engineer-jobs-san-francisco-bay-area](https://www.linkedin.com/jobs/software-engineer-jobs-san-francisco-bay-area)

### **Conclusion & Next Steps**

In this tutorial, you learned how to:
- Set up Tavily web tools (search, extract, crawl with LangChain)
- Build an intelligent web reasearch agent using LangGraph's `create_react_agent`
- Design effective system prompts for autonomous web research
- Test your agent with real-world tasks like product research and job searching

You now have a fully functional web research agent that autonomously combines search, extraction and crawling to complete complex research objectives.